In [1]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import numpy as np
import os
import keras
from keras. preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import concatenate
import scipy

We have to set the directory based on the folder where you saved the  dataset

In [2]:
for dirname, _, filenames in os.walk('C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\'):
    print (dirname)

C:\Users\0MEN\Desktop\sub\resizedto224\
C:\Users\0MEN\Desktop\sub\resizedto224\pred
C:\Users\0MEN\Desktop\sub\resizedto224\Test
C:\Users\0MEN\Desktop\sub\resizedto224\Test\actinic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Test\basal cell carcinoma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\dermatofibroma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\melanoma
C:\Users\0MEN\Desktop\sub\resizedto224\Test\nevus
C:\Users\0MEN\Desktop\sub\resizedto224\Test\pigmented benign keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Test\seborrheic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Train
C:\Users\0MEN\Desktop\sub\resizedto224\Train\actinic keratosis
C:\Users\0MEN\Desktop\sub\resizedto224\Train\basal cell carcinoma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\dermatofibroma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\melanoma
C:\Users\0MEN\Desktop\sub\resizedto224\Train\nevus
C:\Users\0MEN\Desktop\sub\resizedto224\Train\pigmented benign keratosis
C:\Users\0MEN\Desktop\sub\resizedto2

We have to set the directory based on the folder where you saved dataset(Train and Test)

In [3]:
# Importing the data and preprocessing 
from keras.preprocessing.image import ImageDataGenerator 


# set the directory based on the folder where you saved dataset
train_data_dir = "C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\Train\\"
test_data_dir = "C:\\Users\\0MEN\\Desktop\\sub\\resizedto224\\Test\\"
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory=train_data_dir,
                                                 shuffle=True,
                                                 target_size=(224,224), 
                                                 subset="training",
                                                 class_mode='categorical')

val_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory=train_data_dir,
                                                 shuffle=True,
                                                 target_size=(224,224), 
                                                 subset="validation",
                                                 class_mode='categorical')
test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(test_data_dir,
                                       target_size=(224,224), batch_size=32, shuffle=False)


print(train_data.classes)
print(val_data.classes)
print(train_data.class_indices)
print(test_data.classes)
class_names = np.unique(train_data.classes)

Found 1538 images belonging to 7 classes.
Found 381 images belonging to 7 classes.
Found 99 images belonging to 7 classes.
[0 0 0 ... 6 6 6]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6]
{'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 

In [4]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  # Input: 
  # - f1: number of filters of the 1x1 convolutional layer in the first path
  # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
  # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
  # - f4: number of filters of the 1x1 convolutional layer in the fourth path

  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer

We have to change the dense value according to number of classes...

In [5]:
def GoogLeNet():
  # input layer 
  input_layer = Input(shape = (224, 224, 3))

  # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
  X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

  # 2nd Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

  # Extra network 1:
  X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = Flatten()(X1)
  X1 = Dense(1024, activation = 'relu')(X1)
  X1 = Dropout(0.7)(X1)
#change the dense value according to number of classes...
  X1 = Dense(7, activation = 'softmax')(X1)

  
  # 4th Inception block
  X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 5th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 6th Inception block
  X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

  # Extra network 2:
  X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = Flatten()(X2)
  X2 = Dense(1024, activation = 'relu')(X2)
  X2 = Dropout(0.7)(X2)
#change the dense value according to number of classes...
  X2 = Dense(7, activation = 'softmax')(X2)
  
  
  # 7th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

  # 9th Inception block
  X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

  # Global Average pooling layer 
  X = GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = Dropout(0.4)(X)

  # output layer 
#change the dense value according to number of classes...
  X = Dense(7, activation = 'softmax')(X)
  
  # model
  model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

  return model

In [6]:
model=GoogLeNet()

model.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 109, 109, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 54, 54, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 54, 54, 64)   4160        max_pooling2d[0][0]              
__________________________________________________________________________________________

In [7]:
model.compile(optimizer='adam', 
              loss=keras.losses.categorical_crossentropy, 
              metrics=['accuracy'])

In [10]:
hist = model.fit(train_data,
                           validation_data= val_data,
                           epochs=25,verbose=1)

Epoch 1/25
49/49 [==============================] - 10s 199ms/step - loss: 5.3053 - dense_4_loss: 1.7662 - dense_1_loss: 1.7697 - dense_3_loss: 1.7695 - dense_4_accuracy: 0.2321 - dense_1_accuracy: 0.2230 - dense_3_accuracy: 0.2204 - val_loss: 5.2577 - val_dense_4_loss: 1.7553 - val_dense_1_loss: 1.7480 - val_dense_3_loss: 1.7544 - val_dense_4_accuracy: 0.2283 - val_dense_1_accuracy: 0.2283 - val_dense_3_accuracy: 0.2283
Epoch 2/25
49/49 [==============================] - 10s 197ms/step - loss: 5.3064 - dense_4_loss: 1.7720 - dense_1_loss: 1.7663 - dense_3_loss: 1.7681 - dense_4_accuracy: 0.2263 - dense_1_accuracy: 0.2191 - dense_3_accuracy: 0.2464 - val_loss: 5.3214 - val_dense_4_loss: 1.7736 - val_dense_1_loss: 1.7694 - val_dense_3_loss: 1.7784 - val_dense_4_accuracy: 0.2415 - val_dense_1_accuracy: 0.2415 - val_dense_3_accuracy: 0.2415
Epoch 3/25
49/49 [==============================] - 10s 197ms/step - loss: 5.3220 - dense_4_loss: 1.7660 - dense_1_loss: 1.7815 - dense_3_loss: 1.7745

In [11]:
model.evaluate(test_data)

4/4 [==============================] - 0s 80ms/step - loss: 6.0886 - dense_4_loss: 2.0350 - dense_1_loss: 2.0289 - dense_3_loss: 2.0248 - dense_4_accuracy: 0.1616 - dense_1_accuracy: 0.1616 - dense_3_accuracy: 0.1616      


[6.088648319244385,
 2.034975528717041,
 2.028892993927002,
 2.0247802734375,
 0.16161616146564484,
 0.16161616146564484,
 0.16161616146564484]

In [20]:
model.save('C:\\Users\\0MEN\\Desktop\\sub\\googlenet\\intel_googlenet_dec14.h5')